In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

### Import all the required DataSets

In [2]:
part_b = pd.read_csv("provider_data/partb/Part-B.csv",encoding='ISO-8859-1',low_memory=False)
leie_ex = pd.read_csv("provider_data/LEIE_Exclusion.csv",encoding='ISO-8859-1',low_memory=False)

## DATA PRE_PROCESSING -- PART-B (Medicare Physician & Other Practitioners)

In [3]:
part_b

,ï»¿Rndrng_NPI,Rndrng_Prvdr_Last_Org_Name,Rndrng_Prvdr_First_Name,Rndrng_Prvdr_MI,Rndrng_Prvdr_Crdntls,Rndrng_Prvdr_Gndr,Rndrng_Prvdr_Ent_Cd,Rndrng_Prvdr_St1,Rndrng_Prvdr_St2,Rndrng_Prvdr_City,...,Bene_CC_Dprssn_Pct,Bene_CC_Dbts_Pct,Bene_CC_Hyplpdma_Pct,Bene_CC_Hyprtnsn_Pct,Bene_CC_IHD_Pct,Bene_CC_Opo_Pct,Bene_CC_RAOA_Pct,Bene_CC_Sz_Pct,Bene_CC_Strok_Pct,Bene_Avg_Risk_Scre
0,1003000126,Enkeshafi,Ardalan,NaN,M.D.,M,I,6410 Rockledge Dr Ste 304,NaN,Bethesda,...,35.00%,47.00%,73.00%,75.00%,66.00%,11.00%,51.00%,9.00%,19.00%,2.5028
1,1003000134,Cibull,Thomas,L,M.D.,M,I,2650 Ridge Ave,Evanston Hospital,Evanston,...,15.00%,20.0%,48.00%,48.00%,24.00%,10%,38.00%,1.0%,3.00%,1.1124
2,1003000142,Khalil,Rashid,NaN,M.D.,M,I,4126 N Holland Sylvania Rd,Suite 220,Toledo,...,37.00%,35.00%,52.00%,68.00%,31.00%,NaN,75.00%,NaN,NaN,1.6935
3,1003000423,Velotta,Jennifer,A,M.D.,F,I,11100 Euclid Ave,NaN,Cleveland,...,27.00%,NaN,52.00%,50.0%,23.00%,20.0%,32.00%,0%,NaN,0.7089
4,1003000480,Rothchild,Kevin,B,MD,M,I,12605 E 16th Ave,NaN,Aurora,...,43.00%,39.00%,43.00%,60.0%,33.00%,13.00%,57.00%,NaN,NaN,2.1137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1161537,1992999551,Molai,Indira,NaN,M.D.,F,I,625 E Grand Ave,NaN,Escondido,...,28.00%,26.00%,49.00%,66.00%,25.00%,9.00%,41.00%,NaN,4.00%,1.2418
1161538,1992999585,Marek,Norman,F,D.C.,M,I,1355 Four Mile Dr,NaN,Williamsport,...,24.00%,26.00%,61.00%,53.00%,26.00%,NaN,56.00%,NaN,NaN,0.9525
1161539,1992999775,"Spine Surgery Center Of Eugene, Llc",NaN,NaN,NaN,NaN,O,1410 Oak St,Suite 300,Eugene,...,25.00%,17.00%,51.00%,63.00%,21.00%,9.00%,75.00%,NaN,NaN,0.9370
1161540,1992999825,Deschenes,Geoffrey,R,M.D.,M,I,1100 9th Ave,Ms:m4-Pfs,Seattle,...,19.00%,21.00%,44.00%,53.00%,26.00%,9.00%,37.00%,NaN,2.00%,1.1737


In [4]:
part_b["Rndrng_NPI"] = part_b["ï»¿Rndrng_NPI"]
part_b=part_b.drop("ï»¿Rndrng_NPI",axis=1)

In [5]:
np.sort(part_b['Rndrng_NPI'].value_counts())

array([1, 1, 1, ..., 1, 1, 1])

In [6]:
part_b = part_b.dropna(axis=0,thresh=2)
part_b = part_b.dropna(subset=["Rndrng_NPI"],axis=0)
part_b = part_b.dropna(subset=["Rndrng_Prvdr_Last_Org_Name"],axis=0)
part_b = part_b.dropna(subset=["Rndrng_Prvdr_First_Name"],axis=0)

### PRE_PROCESSING OF LEIE DATA

In [7]:
leie_ex = leie_ex.dropna(subset=["NPI"],axis=0)
ex_npi = leie_ex[leie_ex["NPI"]!=0]["NPI"]
ex_lnames = leie_ex.dropna(subset=["LASTNAME"],axis=0)["LASTNAME"]
ex_fnames = leie_ex.dropna(subset=["FIRSTNAME"],axis=0)["FIRSTNAME"]
leie_ex["MIDNAME"] = leie_ex["MIDNAME"].apply(lambda row : row if row!= " " else 0)
ex_mi = leie_ex[leie_ex["MIDNAME"]!=0]["MIDNAME"]

In [8]:
print(leie_ex.shape)
print(part_b.shape)

(76546, 18)
(1100800, 73)


In [9]:
leie_ex["EXCLTYPE"].value_counts()

1128b4       31120
1128a1       21991
1128a2        7318
1128a3        4809
1128a4        3134
1128b14       2243
1128b8        1496
1128a1        1100
1128b1         862
1128b5         807
1128b7         657
1128b3         313
1128Aa         150
1128a3         136
1128a2         112
1128b6          64
1128b2          56
1156            56
1128b15         34
1128b7          24
BRCH SA         17
BRCH CIA        14
1128b11         11
1160             9
1128b6           3
1128a4           3
1128b2           2
1128b16          2
1128b12          1
1128b1           1
1128b5           1
Name: EXCLTYPE, dtype: int64

In [10]:
ex_npi_unique = leie_ex.drop_duplicates("NPI")  # Remove duplicate NPI values
ex_npi_dict = dict(zip(ex_npi_unique['NPI'], ex_npi_unique['EXCLTYPE']))

part_b['Fraud'] = np.where(part_b['Rndrng_NPI'].isin(ex_npi_dict), 1, 0)
part_b['FraudType'] = np.where(part_b['Fraud'] == 1,part_b['Rndrng_NPI'].map(ex_npi_dict),0)

In [11]:
part_b["Fraud"].value_counts()

0    1100684
1        116
Name: Fraud, dtype: int64

In [12]:
part_b["FraudType"].value_counts()

0            1100684
1128b4            35
1128a1            30
1128a4            14
1128a1            13
1128a3             8
1128b7             8
1128b1             2
1128a2             2
BRCH SA            1
1128a2             1
1128b2             1
1128a3             1
Name: FraudType, dtype: int64

In [13]:
part_b['Rndrng_NPI'].value_counts()

1003000126    1
1669673638    1
1669672853    1
1669672960    1
1669672994    1
             ..
1336193614    1
1336193622    1
1336193630    1
1336193648    1
1992999874    1
Name: Rndrng_NPI, Length: 1100800, dtype: int64

### ANALYSIS OF EACH COLUMN

In [14]:
part_b.columns

Index(['Rndrng_Prvdr_Last_Org_Name', 'Rndrng_Prvdr_First_Name',
       'Rndrng_Prvdr_MI', 'Rndrng_Prvdr_Crdntls', 'Rndrng_Prvdr_Gndr',
       'Rndrng_Prvdr_Ent_Cd', 'Rndrng_Prvdr_St1', 'Rndrng_Prvdr_St2',
       'Rndrng_Prvdr_City', 'Rndrng_Prvdr_State_Abrvtn',
       'Rndrng_Prvdr_State_FIPS', 'Rndrng_Prvdr_Zip5', 'Rndrng_Prvdr_RUCA',
       'Rndrng_Prvdr_RUCA_Desc', 'Rndrng_Prvdr_Cntry', 'Rndrng_Prvdr_Type',
       'Rndrng_Prvdr_Mdcr_Prtcptg_Ind', 'Tot_HCPCS_Cds', 'Tot_Benes',
       'Tot_Srvcs', 'Tot_Sbmtd_Chrg', 'Tot_Mdcr_Alowd_Amt',
       'Tot_Mdcr_Pymt_Amt', 'Tot_Mdcr_Stdzd_Amt', 'Drug_Sprsn_Ind',
       'Drug_Tot_HCPCS_Cds', 'Drug_Tot_Benes', 'Drug_Tot_Srvcs',
       'Drug_Sbmtd_Chrg', 'Drug_Mdcr_Alowd_Amt', 'Drug_Mdcr_Pymt_Amt',
       'Drug_Mdcr_Stdzd_Amt', 'Med_Sprsn_Ind', 'Med_Tot_HCPCS_Cds',
       'Med_Tot_Benes', 'Med_Tot_Srvcs', 'Med_Sbmtd_Chrg',
       'Med_Mdcr_Alowd_Amt', 'Med_Mdcr_Pymt_Amt', 'Med_Mdcr_Stdzd_Amt',
       'Bene_Avg_Age', 'Bene_Age_LT_65_Cnt', 'Bene_Ag

In [15]:
part_b.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1100800 entries, 0 to 1161541
Data columns (total 75 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   Rndrng_Prvdr_Last_Org_Name     1100800 non-null  object 
 1   Rndrng_Prvdr_First_Name        1100800 non-null  object 
 2   Rndrng_Prvdr_MI                753740 non-null   object 
 3   Rndrng_Prvdr_Crdntls           1029252 non-null  object 
 4   Rndrng_Prvdr_Gndr              1100800 non-null  object 
 5   Rndrng_Prvdr_Ent_Cd            1100800 non-null  object 
 6   Rndrng_Prvdr_St1               1100800 non-null  object 
 7   Rndrng_Prvdr_St2               314695 non-null   object 
 8   Rndrng_Prvdr_City              1100800 non-null  object 
 9   Rndrng_Prvdr_State_Abrvtn      1100800 non-null  object 
 10  Rndrng_Prvdr_State_FIPS        1100795 non-null  object 
 11  Rndrng_Prvdr_Zip5              1100799 non-null  object 
 12  Rndrng_Prvdr_R

In [16]:
part_b["Rndrng_Prvdr_Crdntls"].value_counts()

M.D.                    251940
MD                      251375
PA-C                     44843
CRNA                     37015
D.O.                     35008
                         ...  
PAC, MPAS, BBA               1
PT, MSPT,  ATC, CSCS         1
DC, DACRB, CSCS              1
RN MS PMHNP                  1
DMD, MD, MHS                 1
Name: Rndrng_Prvdr_Crdntls, Length: 17586, dtype: int64

In [17]:
def convert_to_int(value):
    try:
        return int(value)
    except ValueError:
        return np.nan  

part_b['Rndrng_Prvdr_Zip5'] = part_b['Rndrng_Prvdr_Zip5'].apply(convert_to_int)


In [18]:
part_b = part_b[part_b["Rndrng_Prvdr_Cntry"] == 'US']

### Rndrng_Prvdr_Type

In [19]:
# No Null
part_b[part_b['Rndrng_Prvdr_Type'].isnull()]["Fraud"].value_counts()

Series([], Name: Fraud, dtype: int64)

In [20]:
# What are the values Present
part_b["Rndrng_Prvdr_Type"].value_counts()

Nurse Practitioner                        149318
Internal Medicine                          91092
Physician Assistant                        86288
Family Practice                            83394
Physical Therapist in Private Practice     59414
                                           ...  
Portable X-Ray Supplier                        4
Clinic or Group Practice                       2
Adult Congenital Heart Disease                 1
Slide Preparation Facility                     1
Pharmacy                                       1
Name: Rndrng_Prvdr_Type, Length: 93, dtype: int64

In [21]:
# PreProcessing Scenerio -> Label Encoder (do it while Modelling)

### Rndrng_Prvdr_Mdcr_Prtcptg_Ind

In [22]:
# No Null
part_b[part_b['Rndrng_Prvdr_Mdcr_Prtcptg_Ind'].isnull()]["Fraud"].value_counts()

Series([], Name: Fraud, dtype: int64)

In [23]:
# What are the values Present
part_b["Rndrng_Prvdr_Mdcr_Prtcptg_Ind"].value_counts()

Y    1099762
N        964
Name: Rndrng_Prvdr_Mdcr_Prtcptg_Ind, dtype: int64

In [24]:
# PreProcessing Scenerio
part_b["Rndrng_Prvdr_Mdcr_Prtcptg_Ind"] = part_b["Rndrng_Prvdr_Mdcr_Prtcptg_Ind"].apply(lambda row : 1 if row == 'Y' else 0)

In [25]:
# What are the values Present
part_b["Rndrng_Prvdr_Mdcr_Prtcptg_Ind"].value_counts()

1    1099762
0        964
Name: Rndrng_Prvdr_Mdcr_Prtcptg_Ind, dtype: int64

### Tot_HCPCS_Cds

In [26]:
# No Null
part_b[part_b['Tot_HCPCS_Cds'].isnull()]["Fraud"].value_counts()

Series([], Name: Fraud, dtype: int64)

In [27]:
# What are the values Present
part_b["Tot_HCPCS_Cds"].value_counts()

9      36974
10     36665
8      36405
11     35310
7      34641
       ...  
411        1
378        1
331        1
299        1
349        1
Name: Tot_HCPCS_Cds, Length: 328, dtype: int64

In [28]:
# PreProcessing Scenerio
part_b["Tot_HCPCS_Cds"] = part_b["Tot_HCPCS_Cds"].astype(float)

In [29]:
# What are the values Present
part_b["Tot_HCPCS_Cds"].value_counts()

9.0      36974
10.0     36665
8.0      36405
11.0     35310
7.0      34641
         ...  
411.0        1
378.0        1
331.0        1
299.0        1
349.0        1
Name: Tot_HCPCS_Cds, Length: 328, dtype: int64

### Tot_Benes

In [30]:
# No Null
part_b[part_b['Tot_Benes'].isnull()]["Fraud"].value_counts()

Series([], Name: Fraud, dtype: int64)

In [31]:
# What are the values Present
part_b["Tot_Benes"].value_counts()

11       10756
12       10254
13        9791
14        9235
15        8931
         ...  
7,001        1
5,938        1
4,688        1
9,595        1
6,922        1
Name: Tot_Benes, Length: 5512, dtype: int64

In [32]:
part_b["Tot_Benes"] = part_b["Tot_Benes"].str.replace(',', '').astype(float)

In [33]:
# What are the values Present
part_b["Tot_Benes"].value_counts()

11.0      10756
12.0      10254
13.0       9791
14.0       9235
15.0       8931
          ...  
7001.0        1
5938.0        1
4688.0        1
9595.0        1
6922.0        1
Name: Tot_Benes, Length: 5512, dtype: int64

### Tot_Srvcs

In [34]:
# No Null
part_b[part_b['Tot_Srvcs'].isnull()]["Fraud"].value_counts()

Series([], Name: Fraud, dtype: int64)

In [35]:
# What are the values Present
part_b["Tot_Srvcs"].value_counts()

23         2508
29         2501
26         2499
21         2478
24         2472
           ... 
25,600        1
23,575        1
18,986        1
292,161       1
76,925        1
Name: Tot_Srvcs, Length: 30337, dtype: int64

In [37]:
part_b["Tot_Srvcs"] = part_b["Tot_Srvcs"].replace(',','',regex=True).astype(float)

In [38]:
# What are the values Present
part_b["Tot_Srvcs"].value_counts()

23.0        2508
29.0        2501
26.0        2499
21.0        2478
24.0        2472
            ... 
25600.0        1
23575.0        1
18986.0        1
292161.0       1
76925.0        1
Name: Tot_Srvcs, Length: 30337, dtype: int64

### Tot_Sbmtd_Chrg

In [39]:
# No Null
part_b[part_b['Tot_Sbmtd_Chrg'].isnull()]["Fraud"].value_counts()

Series([], Name: Fraud, dtype: int64)

In [40]:
# What are the values Present
part_b["Tot_Sbmtd_Chrg"].value_counts()

$11,700.00     47
$4,800.00      45
$10,800.00     43
$7,200.00      42
$5,400.00      42
               ..
$4,613.65       1
$303,229.01     1
$71,450.42      1
$352,690.00     1
$340,479.02     1
Name: Tot_Sbmtd_Chrg, Length: 700974, dtype: int64

In [41]:
part_b["Tot_Sbmtd_Chrg"] = part_b["Tot_Sbmtd_Chrg"].replace('[\$,]', '', regex=True).astype(float)

In [42]:
# What are the values Present
part_b["Tot_Sbmtd_Chrg"].value_counts()

11700.00     47
4800.00      45
10800.00     43
7200.00      42
5400.00      42
             ..
4613.65       1
303229.01     1
71450.42      1
352690.00     1
340479.02     1
Name: Tot_Sbmtd_Chrg, Length: 700974, dtype: int64

### Tot_Mdcr_Alowd_Amt

In [43]:
# No Null
part_b[part_b['Tot_Mdcr_Alowd_Amt'].isnull()]["Fraud"].value_counts()

Series([], Name: Fraud, dtype: int64)

In [44]:
part_b["Tot_Mdcr_Alowd_Amt"].value_counts()

$2,054.38     5
$1,896.14     5
$1,719.01     5
$12,577.62    5
$4,409.28     5
             ..
$38,676.80    1
$48,328.60    1
$2,119.76     1
$14,792.19    1
$90,924.61    1
Name: Tot_Mdcr_Alowd_Amt, Length: 1045504, dtype: int64

In [45]:
part_b["Tot_Mdcr_Alowd_Amt"] = part_b["Tot_Mdcr_Alowd_Amt"].replace('[\$,]', '', regex=True).astype(float)

In [46]:
part_b["Tot_Mdcr_Alowd_Amt"].value_counts()

2054.38     5
1896.14     5
1719.01     5
12577.62    5
4409.28     5
           ..
38676.80    1
48328.60    1
2119.76     1
14792.19    1
90924.61    1
Name: Tot_Mdcr_Alowd_Amt, Length: 1045504, dtype: int64

### Tot_Mdcr_Pymt_Amt

In [47]:
# No Null
part_b[part_b['Tot_Mdcr_Pymt_Amt'].isnull()]["Fraud"].value_counts()

Series([], Name: Fraud, dtype: int64)

In [48]:
part_b["Tot_Mdcr_Pymt_Amt"].value_counts()

$2,083.00     6
$0.00         6
$3,679.20     6
$2,482.21     6
$5,681.52     5
             ..
$10,189.28    1
$4,966.44     1
$4,439.23     1
$1,314.94     1
$74,063.76    1
Name: Tot_Mdcr_Pymt_Amt, Length: 1028450, dtype: int64

In [49]:
part_b["Tot_Mdcr_Pymt_Amt"] = part_b["Tot_Mdcr_Pymt_Amt"].replace('[\$,]', '', regex=True).astype(float)

In [50]:
part_b["Tot_Mdcr_Pymt_Amt"].value_counts()

2083.00     6
0.00        6
3679.20     6
2482.21     6
5681.52     5
           ..
10189.28    1
4966.44     1
4439.23     1
1314.94     1
74063.76    1
Name: Tot_Mdcr_Pymt_Amt, Length: 1028450, dtype: int64

### Tot_Mdcr_Stdzd_Amt

In [51]:
# No Null
part_b[part_b['Tot_Mdcr_Stdzd_Amt'].isnull()]["Fraud"].value_counts()

Series([], Name: Fraud, dtype: int64)

In [52]:
part_b["Tot_Mdcr_Stdzd_Amt"].value_counts()

$3,026.22     7
$1,571.58     6
$842.18       6
$918.42       6
$788.17       5
             ..
$292.60       1
$524.07       1
$41,776.66    1
$11,535.81    1
$73,646.68    1
Name: Tot_Mdcr_Stdzd_Amt, Length: 1028263, dtype: int64

In [53]:
part_b["Tot_Mdcr_Stdzd_Amt"] = part_b["Tot_Mdcr_Stdzd_Amt"].replace('[\$,]', '', regex=True).astype(float)

In [54]:
part_b["Tot_Mdcr_Stdzd_Amt"].value_counts()

3026.22     7
1571.58     6
842.18      6
918.42      6
788.17      5
           ..
292.60      1
524.07      1
41776.66    1
11535.81    1
73646.68    1
Name: Tot_Mdcr_Stdzd_Amt, Length: 1028263, dtype: int64

### Drug_Sprsn_Ind

In [55]:
# Null Values Present use Imputation
part_b[part_b['Drug_Sprsn_Ind'].isnull()]["Fraud"].value_counts()

0    969933
1        93
Name: Fraud, dtype: int64

In [56]:
part_b["Drug_Sprsn_Ind"].value_counts()

*    130659
#        41
Name: Drug_Sprsn_Ind, dtype: int64

In [57]:
replace_dict = {'#': 1, '*': 0, 'NA': pd.NA}
part_b['Drug_Sprsn_Ind'] = part_b['Drug_Sprsn_Ind'].replace(replace_dict)

In [58]:
part_b["Drug_Sprsn_Ind"].value_counts()

0.0    130659
1.0        41
Name: Drug_Sprsn_Ind, dtype: int64

### Drug_Tot_HCPCS_Cds

In [59]:
# Null Values Present use Imputation
part_b[part_b['Drug_Tot_HCPCS_Cds'].isnull()]["Fraud"].value_counts()

0    130677
1        23
Name: Fraud, dtype: int64

In [60]:
part_b["Drug_Tot_HCPCS_Cds"].value_counts()

0.0      755115
5.0       24169
4.0       24069
2.0       23847
3.0       23722
          ...  
127.0         1
129.0         1
131.0         1
119.0         1
126.0         1
Name: Drug_Tot_HCPCS_Cds, Length: 131, dtype: int64

In [61]:
part_b["Drug_Tot_HCPCS_Cds"] = part_b["Drug_Tot_HCPCS_Cds"].replace('[\$,]', '', regex=True).astype(float)

In [62]:
part_b["Drug_Tot_HCPCS_Cds"].value_counts()

0.0      755115
5.0       24169
4.0       24069
2.0       23847
3.0       23722
          ...  
127.0         1
129.0         1
131.0         1
119.0         1
126.0         1
Name: Drug_Tot_HCPCS_Cds, Length: 131, dtype: int64

### Drug_Tot_Benes

In [63]:
# Null Values Present use Imputation
part_b[part_b['Drug_Tot_Benes'].isnull()]["Fraud"].value_counts()

0    130677
1        23
Name: Fraud, dtype: int64

In [64]:
part_b["Drug_Tot_Benes"].value_counts()

0        755115
11         5951
12         5429
13         5035
14         4828
          ...  
1,125         1
796           1
751           1
959           1
1,106         1
Name: Drug_Tot_Benes, Length: 967, dtype: int64

In [65]:
part_b["Drug_Tot_Benes"] = part_b["Drug_Tot_Benes"].replace(',', '', regex=True).astype(float)

In [66]:
part_b["Drug_Tot_Benes"]

0           0.0
1           0.0
2           NaN
3           NaN
4           0.0
           ... 
1161536     0.0
1161537    78.0
1161538     0.0
1161540     NaN
1161541     0.0
Name: Drug_Tot_Benes, Length: 1100726, dtype: float64

### Drug_Tot_Srvcs

In [67]:
# Null Values Present use Imputation
part_b[part_b['Drug_Tot_Srvcs'].isnull()]["Fraud"].value_counts()

0    130677
1        23
Name: Fraud, dtype: int64

In [68]:
part_b["Drug_Tot_Srvcs"].value_counts()

0          755115
17           1492
19           1455
22           1448
20           1433
            ...  
97,816          1
14,226          1
28,164          1
333,340         1
15,087          1
Name: Drug_Tot_Srvcs, Length: 23743, dtype: int64

In [69]:
part_b["Drug_Tot_Srvcs"] = part_b["Drug_Tot_Srvcs"].str.replace(',','').astype(float)

In [70]:
part_b["Drug_Tot_Srvcs"].value_counts()

0.0         755115
17.0          1492
19.0          1455
22.0          1448
20.0          1433
             ...  
97816.0          1
14226.0          1
28164.0          1
333340.0         1
15087.0          1
Name: Drug_Tot_Srvcs, Length: 23743, dtype: int64

### Drug_Sbmtd_Chrg

In [71]:
# Null Values Present use Imputation
part_b[part_b['Drug_Sbmtd_Chrg'].isnull()]["Fraud"].value_counts()

0    130677
1        23
Name: Fraud, dtype: int64

In [72]:
part_b["Drug_Sbmtd_Chrg"].value_counts()

$0.00            755115
$600.00              99
$480.00              90
$840.00              90
$900.00              86
                  ...  
$6,726,867.68         1
$264.66               1
$76,925.00            1
$69,713.02            1
$1,077,270.00         1
Name: Drug_Sbmtd_Chrg, Length: 114350, dtype: int64

In [73]:
part_b["Drug_Sbmtd_Chrg"] = part_b["Drug_Sbmtd_Chrg"].replace('[\$,]', '', regex=True).astype(float)

In [74]:
part_b["Drug_Sbmtd_Chrg"]

0              0.0
1              0.0
2              NaN
3              NaN
4              0.0
            ...   
1161536        0.0
1161537    11406.0
1161538        0.0
1161540        NaN
1161541        0.0
Name: Drug_Sbmtd_Chrg, Length: 1100726, dtype: float64

### Drug_Mdcr_Alowd_Amt

In [75]:
# Null Values Present use Imputation
part_b[part_b['Drug_Mdcr_Alowd_Amt'].isnull()]["Fraud"].value_counts()

0    130677
1        23
Name: Fraud, dtype: int64

In [76]:
part_b["Drug_Mdcr_Alowd_Amt"].value_counts()

$0.00         755115
$670.78           34
$731.76           23
$215.38           21
$304.70           17
               ...  
$2,263.72          1
$29,729.90         1
$3,661.88          1
$4,330.51          1
$7,150.38          1
Name: Drug_Mdcr_Alowd_Amt, Length: 192989, dtype: int64

In [77]:
part_b["Drug_Mdcr_Alowd_Amt"] = part_b["Drug_Mdcr_Alowd_Amt"].replace('[\$,]', '', regex=True).astype(float)

In [81]:
# Null Values Present use Imputation
part_b['Drug_Mdcr_Alowd_Amt']

0             0.00
1             0.00
2              NaN
3              NaN
4             0.00
            ...   
1161536       0.00
1161537    7150.38
1161538       0.00
1161540        NaN
1161541       0.00
Name: Drug_Mdcr_Alowd_Amt, Length: 1100726, dtype: float64

### Drug_Mdcr_Pymt_Amt

In [82]:
# Null Values Present use Imputation
part_b[part_b['Drug_Mdcr_Pymt_Amt'].isnull()]["Fraud"].value_counts()

0    130677
1        23
Name: Fraud, dtype: int64

In [83]:
part_b["Drug_Mdcr_Pymt_Amt"].value_counts()

$0.00         755116
$670.78           34
$731.76           23
$215.38           21
$914.70           19
               ...  
$26,820.17         1
$566.90            1
$4,992.97          1
$745.54            1
$7,146.82          1
Name: Drug_Mdcr_Pymt_Amt, Length: 190517, dtype: int64

In [84]:
part_b["Drug_Mdcr_Pymt_Amt"] = part_b["Drug_Mdcr_Pymt_Amt"].replace('[\$,]', '', regex=True).astype(float)

In [85]:
part_b["Drug_Mdcr_Pymt_Amt"]

0             0.00
1             0.00
2              NaN
3              NaN
4             0.00
            ...   
1161536       0.00
1161537    7146.82
1161538       0.00
1161540        NaN
1161541       0.00
Name: Drug_Mdcr_Pymt_Amt, Length: 1100726, dtype: float64

### Drug_Mdcr_Stdzd_Amt

In [86]:
# Null Values Present use Imputation
part_b[part_b['Drug_Mdcr_Stdzd_Amt'].isnull()]["Fraud"].value_counts()

0    130677
1        23
Name: Fraud, dtype: int64

In [87]:
part_b["Drug_Mdcr_Stdzd_Amt"].value_counts()

$0.00         755116
$657.36           30
$717.12           20
$211.09           18
$776.88           17
               ...  
$32,598.47         1
$28,457.63         1
$787.64            1
$4,893.30          1
$7,041.10          1
Name: Drug_Mdcr_Stdzd_Amt, Length: 190451, dtype: int64

In [88]:
part_b["Drug_Mdcr_Stdzd_Amt"] = part_b["Drug_Mdcr_Stdzd_Amt"].replace('[\$,]', '', regex=True).astype(float)

In [89]:
part_b["Drug_Mdcr_Stdzd_Amt"]

0             0.0
1             0.0
2             NaN
3             NaN
4             0.0
            ...  
1161536       0.0
1161537    7041.1
1161538       0.0
1161540       NaN
1161541       0.0
Name: Drug_Mdcr_Stdzd_Amt, Length: 1100726, dtype: float64

###  Med_Sprsn_Ind 

In [90]:
# Null Values Present use Imputation
part_b[part_b['Med_Sprsn_Ind'].isnull()]["Fraud"].value_counts()

0    969933
1        93
Name: Fraud, dtype: int64

In [91]:
part_b["Med_Sprsn_Ind"].value_counts()

#    130480
*       220
Name: Med_Sprsn_Ind, dtype: int64

In [92]:
replace_dict = {'#': 1, '*': 0, 'NA': pd.NA}
part_b['Med_Sprsn_Ind'] = part_b['Med_Sprsn_Ind'].replace(replace_dict)

In [93]:
part_b["Med_Sprsn_Ind"]

0          NaN
1          NaN
2          1.0
3          1.0
4          NaN
          ... 
1161536    NaN
1161537    NaN
1161538    NaN
1161540    1.0
1161541    NaN
Name: Med_Sprsn_Ind, Length: 1100726, dtype: float64

### Med_Tot_HCPCS_Cds

In [94]:
# Null Values Present use Imputation
part_b[part_b['Med_Tot_HCPCS_Cds'].isnull()]["Fraud"].value_counts()

0    130677
1        23
Name: Fraud, dtype: int64

In [95]:
part_b["Med_Tot_HCPCS_Cds"].value_counts()

9.0      35224
8.0      34854
10.0     34570
7.0      33475
11.0     32966
         ...  
623.0        1
334.0        1
378.0        1
411.0        1
330.0        1
Name: Med_Tot_HCPCS_Cds, Length: 323, dtype: int64

In [96]:
part_b["Med_Tot_HCPCS_Cds"] = part_b["Med_Tot_HCPCS_Cds"].astype(float)

In [97]:
part_b["Med_Tot_HCPCS_Cds"]

0          16.0
1          18.0
2           NaN
3           NaN
4          30.0
           ... 
1161536     9.0
1161537    29.0
1161538     2.0
1161540     NaN
1161541    15.0
Name: Med_Tot_HCPCS_Cds, Length: 1100726, dtype: float64

### Med_Tot_Benes

In [98]:
# Null Values Present use Imputation
part_b[part_b['Med_Tot_Benes'].isnull()]["Fraud"].value_counts()

0    130677
1        23
Name: Fraud, dtype: int64

In [99]:
part_b["Med_Tot_Benes"].value_counts()

11       9730
12       9168
13       8790
14       8250
15       7978
         ... 
4,986       1
4,346       1
7,722       1
4,372       1
6,919       1
Name: Med_Tot_Benes, Length: 5406, dtype: int64

In [100]:
part_b["Med_Tot_Benes"] = part_b["Med_Tot_Benes"].replace(',', '', regex=True).astype(float)

In [101]:
part_b["Med_Tot_Benes"]

0           291.0
1          2633.0
2             NaN
3             NaN
4            89.0
            ...  
1161536     155.0
1161537     333.0
1161538      66.0
1161540       NaN
1161541     407.0
Name: Med_Tot_Benes, Length: 1100726, dtype: float64

### Med_Tot_Srvcs

In [102]:
# Null Values Present use Imputation
part_b[part_b['Med_Tot_Srvcs'].isnull()]["Fraud"].value_counts()

0    130677
1        23
Name: Fraud, dtype: int64

In [103]:
part_b["Med_Tot_Srvcs"].value_counts()

20        2256
23        2250
21        2237
29        2233
22        2217
          ... 
14,615       1
24,502       1
21,673       1
18,283       1
9,870        1
Name: Med_Tot_Srvcs, Length: 15095, dtype: int64

In [104]:
part_b["Med_Tot_Srvcs"] = part_b["Med_Tot_Srvcs"].replace(',', '', regex=True).astype(float)

In [105]:
part_b["Med_Tot_Srvcs"]

0           764.0
1          5930.0
2             NaN
3             NaN
4           125.0
            ...  
1161536     171.0
1161537     976.0
1161538     415.0
1161540       NaN
1161541     884.0
Name: Med_Tot_Srvcs, Length: 1100726, dtype: float64

### Med_Sbmtd_Chrg

In [106]:
# Null Values Present use Imputation
part_b[part_b['Med_Sbmtd_Chrg'].isnull()]["Fraud"].value_counts()

0    130677
1        23
Name: Fraud, dtype: int64

In [107]:
part_b["Med_Sbmtd_Chrg"].value_counts()

$4,800.00      44
$7,200.00      42
$11,700.00     42
$10,800.00     39
$5,400.00      38
               ..
$339,175.00     1
$214,315.80     1
$460,429.00     1
$100,343.56     1
$340,479.02     1
Name: Med_Sbmtd_Chrg, Length: 612522, dtype: int64

In [108]:
part_b["Med_Sbmtd_Chrg"] = part_b["Med_Sbmtd_Chrg"].replace('[\$,]', '', regex=True).astype(float)

In [109]:
part_b["Med_Sbmtd_Chrg"]

0          402812.00
1          915291.00
2                NaN
3                NaN
4          152154.00
             ...    
1161536    329504.00
1161537    171109.00
1161538     20985.00
1161540          NaN
1161541    340479.02
Name: Med_Sbmtd_Chrg, Length: 1100726, dtype: float64

### Med_Mdcr_Alowd_Amt

In [110]:
# Null Values Present use Imputation
part_b[part_b['Med_Mdcr_Alowd_Amt'].isnull()]["Fraud"].value_counts()

0    130677
1        23
Name: Fraud, dtype: int64

In [111]:
part_b["Med_Mdcr_Alowd_Amt"].value_counts()

$0.00         9
$1,896.14     5
$4,193.13     5
$6,992.11     5
$13,006.76    5
             ..
$29,822.62    1
$49,441.66    1
$645.29       1
$11,029.16    1
$90,924.61    1
Name: Med_Mdcr_Alowd_Amt, Length: 927407, dtype: int64

In [112]:
part_b["Med_Mdcr_Alowd_Amt"] = part_b["Med_Mdcr_Alowd_Amt"].replace('[\$,]', '', regex=True).astype(float)

In [113]:
part_b["Med_Mdcr_Alowd_Amt"]

0           85319.63
1          227372.53
2                NaN
3                NaN
4           30631.10
             ...    
1161536     32266.94
1161537     86354.76
1161538     16171.02
1161540          NaN
1161541     90924.61
Name: Med_Mdcr_Alowd_Amt, Length: 1100726, dtype: float64

### Med_Mdcr_Pymt_Amt

In [114]:
# Null Values Present use Imputation
part_b[part_b['Med_Mdcr_Pymt_Amt'].isnull()]["Fraud"].value_counts()

0    130677
1        23
Name: Fraud, dtype: int64

In [115]:
part_b["Med_Mdcr_Pymt_Amt"].value_counts()

$0.00         15
$2,083.00      6
$2,482.21      6
$3,268.47      5
$585.06        5
              ..
$59,853.56     1
$5,138.15      1
$89,594.61     1
$10,366.00     1
$74,063.76     1
Name: Med_Mdcr_Pymt_Amt, Length: 913985, dtype: int64

In [116]:
part_b["Med_Mdcr_Pymt_Amt"] = part_b["Med_Mdcr_Pymt_Amt"].replace('[\$,]', '', regex=True).astype(float)

In [117]:
part_b["Med_Mdcr_Pymt_Amt"]

0           69175.78
1          176497.74
2                NaN
3                NaN
4           23962.85
             ...    
1161536     25803.90
1161537     61105.21
1161538     11759.42
1161540          NaN
1161541     74063.76
Name: Med_Mdcr_Pymt_Amt, Length: 1100726, dtype: float64

### Med_Mdcr_Stdzd_Amt

In [118]:
# Null Values Present use Imputation
part_b[part_b['Med_Mdcr_Stdzd_Amt'].isnull()]["Fraud"].value_counts()

0    130677
1        23
Name: Fraud, dtype: int64

In [119]:
part_b["Med_Mdcr_Stdzd_Amt"].value_counts()

$0.00         12
$3,026.22      7
$4,121.52      5
$566.00        5
$6,045.22      5
              ..
$11,072.90     1
$897.06        1
$3,599.82      1
$3,023.17      1
$73,646.68     1
Name: Med_Mdcr_Stdzd_Amt, Length: 913964, dtype: int64

In [120]:
part_b["Med_Mdcr_Stdzd_Amt"] = part_b["Med_Mdcr_Stdzd_Amt"].replace('[\$,]', '', regex=True).astype(float)

In [121]:
part_b["Med_Mdcr_Stdzd_Amt"]

0           66401.61
1          167363.18
2                NaN
3                NaN
4           23463.98
             ...    
1161536     22622.59
1161537     55630.65
1161538     11851.98
1161540          NaN
1161541     73646.68
Name: Med_Mdcr_Stdzd_Amt, Length: 1100726, dtype: float64

### Bene_Avg_Age

In [122]:
# Null Values Present use Imputation
part_b[part_b['Bene_Avg_Age'].isnull()]["Fraud"].value_counts()

Series([], Name: Fraud, dtype: int64)

In [123]:
part_b["Bene_Avg_Age"].value_counts()

73    142154
72    137102
74    120544
71    112759
75     87965
       ...  
92         8
21         7
9          5
94         1
8          1
Name: Bene_Avg_Age, Length: 86, dtype: int64

In [125]:
part_b["Bene_Avg_Age"] = part_b["Bene_Avg_Age"].astype(float)

In [126]:
part_b["Bene_Avg_Age"]

0          77.0
1          76.0
2          65.0
3          67.0
4          64.0
           ... 
1161536    77.0
1161537    76.0
1161538    71.0
1161540    74.0
1161541    75.0
Name: Bene_Avg_Age, Length: 1100726, dtype: float64

### Bene_Age_LT_65_Cnt

In [127]:
# Null Values Present use Imputation
part_b[part_b['Bene_Age_LT_65_Cnt'].isnull()]["Fraud"].value_counts()

0    463045
1        48
Name: Fraud, dtype: int64

In [128]:
part_b["Bene_Age_LT_65_Cnt"].value_counts()

0        16734
15       14298
14       14135
13       14050
16       13995
         ...  
1,486        1
1,176        1
986          1
989          1
2,982        1
Name: Bene_Age_LT_65_Cnt, Length: 1153, dtype: int64

In [129]:
part_b["Bene_Age_LT_65_Cnt"] = part_b["Bene_Age_LT_65_Cnt"].replace(',', '', regex=True).astype(float)

In [131]:
part_b["Bene_Age_LT_65_Cnt"]

0          27.0
1          69.0
2          70.0
3          12.0
4          34.0
           ... 
1161536    21.0
1161537    21.0
1161538     NaN
1161540    33.0
1161541    47.0
Name: Bene_Age_LT_65_Cnt, Length: 1100726, dtype: float64

### Bene_Age_65_74_Cnt

In [132]:
# Null Values Present use Imputation
part_b[part_b['Bene_Age_65_74_Cnt'].isnull()]["Fraud"].value_counts()

0    192046
1        37
Name: Fraud, dtype: int64

In [133]:
part_b["Bene_Age_65_74_Cnt"].value_counts()

25       8425
22       8402
24       8380
23       8352
20       8349
         ... 
3,235       1
3,463       1
1,967       1
2,597       1
2,406       1
Name: Bene_Age_65_74_Cnt, Length: 2818, dtype: int64

In [134]:
part_b["Bene_Age_65_74_Cnt"] = part_b["Bene_Age_65_74_Cnt"].replace(',', '', regex=True).astype(float)

In [135]:
part_b["Bene_Age_65_74_Cnt"]

0            88.0
1          1189.0
2            61.0
3            33.0
4            43.0
            ...  
1161536      39.0
1161537     129.0
1161538      37.0
1161540     205.0
1161541     133.0
Name: Bene_Age_65_74_Cnt, Length: 1100726, dtype: float64

### Bene_Age_75_84_Cnt

In [136]:
# Null Values Present use Imputation
part_b[part_b['Bene_Age_75_84_Cnt'].isnull()]["Fraud"].value_counts()

0    342438
1        53
Name: Fraud, dtype: int64

In [137]:
part_b["Bene_Age_75_84_Cnt"].value_counts()

25       8221
26       8210
27       8179
30       8080
28       8078
         ... 
2,092       1
2,775       1
3,776       1
2,173       1
2,559       1
Name: Bene_Age_75_84_Cnt, Length: 2214, dtype: int64

In [138]:
part_b["Bene_Age_75_84_Cnt"] = part_b["Bene_Age_75_84_Cnt"].replace(',', '', regex=True).astype(float)

In [139]:
part_b["Bene_Age_75_84_Cnt"]

0          104.0
1          963.0
2            NaN
3            NaN
4            NaN
           ...  
1161536     52.0
1161537    111.0
1161538     17.0
1161540    152.0
1161541    132.0
Name: Bene_Age_75_84_Cnt, Length: 1100726, dtype: float64

### Bene_Age_GT_84_Cnt

In [140]:
# Null Values Present use Imputation
part_b[part_b['Bene_Age_GT_84_Cnt'].isnull()]["Fraud"].value_counts()

0    505797
1        51
Name: Fraud, dtype: int64

In [141]:
part_b["Bene_Age_GT_84_Cnt"].value_counts()

0        43408
11       15762
12       15080
13       14659
14       13876
         ...  
1,573        1
1,264        1
976          1
1,246        1
1,157        1
Name: Bene_Age_GT_84_Cnt, Length: 1261, dtype: int64

In [142]:
part_b["Bene_Age_GT_84_Cnt"] = part_b["Bene_Age_GT_84_Cnt"].replace(',', '', regex=True).astype(float)

In [143]:
part_b["Bene_Age_GT_84_Cnt"]

0           72.0
1          412.0
2            NaN
3            NaN
4            NaN
           ...  
1161536     43.0
1161537     72.0
1161538      NaN
1161540     66.0
1161541     95.0
Name: Bene_Age_GT_84_Cnt, Length: 1100726, dtype: float64

### Bene_Feml_Cnt

In [144]:
# Null Values Present use Imputation
part_b[part_b['Bene_Feml_Cnt'].isnull()]["Fraud"].value_counts()

0    140744
1        23
Name: Fraud, dtype: int64

In [145]:
part_b["Bene_Feml_Cnt"].value_counts()

23       7560
22       7466
26       7459
25       7455
29       7439
         ... 
3,257       1
4,027       1
2,847       1
3,628       1
2,936       1
Name: Bene_Feml_Cnt, Length: 3669, dtype: int64

In [146]:
part_b["Bene_Feml_Cnt"] = part_b["Bene_Feml_Cnt"].replace(',', '', regex=True).astype(float)

In [147]:
part_b["Bene_Feml_Cnt"]

0           161.0
1          1310.0
2           100.0
3            56.0
4            48.0
            ...  
1161536      85.0
1161537     243.0
1161538      33.0
1161540     242.0
1161541     202.0
Name: Bene_Feml_Cnt, Length: 1100726, dtype: float64

### Bene_Male_Cnt

In [148]:
# Null Values Present use Imputation
part_b[part_b['Bene_Male_Cnt'].isnull()]["Fraud"].value_counts()

0    140744
1        23
Name: Fraud, dtype: int64

In [149]:
part_b["Bene_Male_Cnt"].value_counts()

0        31707
13       10740
14       10683
15       10605
16       10573
         ...  
2,461        1
3,736        1
3,445        1
2,315        1
3,383        1
Name: Bene_Male_Cnt, Length: 2807, dtype: int64

In [150]:
part_b["Bene_Male_Cnt"] = part_b["Bene_Male_Cnt"].replace(',', '', regex=True).astype(float)

In [151]:
part_b["Bene_Male_Cnt"]

0           130.0
1          1323.0
2            67.0
3             0.0
4            41.0
            ...  
1161536      70.0
1161537      90.0
1161538      33.0
1161540     214.0
1161541     205.0
Name: Bene_Male_Cnt, Length: 1100726, dtype: float64

### Bene_Race_Wht_Cnt

In [152]:
# Null Values Present use Imputation
part_b[part_b['Bene_Race_Wht_Cnt'].isnull()]["Fraud"].value_counts()

0    379126
1        46
Name: Fraud, dtype: int64

In [153]:
part_b["Bene_Race_Wht_Cnt"].value_counts()

11        3218
12        3173
13        3028
14        2834
15        2795
          ... 
4,513        1
3,956        1
6,329        1
11,646       1
6,381        1
Name: Bene_Race_Wht_Cnt, Length: 4869, dtype: int64

In [154]:
part_b["Bene_Race_Wht_Cnt"] = part_b["Bene_Race_Wht_Cnt"].replace(',', '', regex=True).astype(float)

In [155]:
part_b["Bene_Race_Wht_Cnt"]

0           210.0
1          2385.0
2           115.0
3             NaN
4            68.0
            ...  
1161536       NaN
1161537     274.0
1161538       NaN
1161540     399.0
1161541     305.0
Name: Bene_Race_Wht_Cnt, Length: 1100726, dtype: float64

### Bene_Race_Black_Cnt

In [156]:
# Null Values Present use Imputation
part_b[part_b['Bene_Race_Black_Cnt'].isnull()]["Fraud"].value_counts()

0    751158
1        73
Name: Fraud, dtype: int64

In [157]:
part_b["Bene_Race_Black_Cnt"].value_counts()

0        52074
12        7898
13        7878
11        7751
14        7454
         ...  
1,076        1
1,577        1
1,573        1
1,926        1
1,320        1
Name: Bene_Race_Black_Cnt, Length: 1429, dtype: int64

In [158]:
part_b["Bene_Race_Black_Cnt"] = part_b["Bene_Race_Black_Cnt"].replace(',', '', regex=True).astype(float)

In [159]:
part_b["Bene_Race_Black_Cnt"]

0          50.0
1           NaN
2          39.0
3           NaN
4           NaN
           ... 
1161536     NaN
1161537     NaN
1161538     NaN
1161540     NaN
1161541    89.0
Name: Bene_Race_Black_Cnt, Length: 1100726, dtype: float64

### Bene_Race_API_Cnt

In [160]:
# Null Values Present use Imputation
part_b[part_b['Bene_Race_API_Cnt'].isnull()]["Fraud"].value_counts()

0    910549
1        91
Name: Fraud, dtype: int64

In [161]:
part_b["Bene_Race_API_Cnt"].value_counts()

0        106099
11         4622
12         4438
13         4065
14         3879
          ...  
1,207         1
680           1
1,023         1
845           1
647           1
Name: Bene_Race_API_Cnt, Length: 788, dtype: int64

In [162]:
part_b["Bene_Race_API_Cnt"] = part_b["Bene_Race_API_Cnt"].replace(',', '', regex=True).astype(float)

In [163]:
part_b["Bene_Race_API_Cnt"]

0           NaN
1          45.0
2           0.0
3           NaN
4           NaN
           ... 
1161536     NaN
1161537    16.0
1161538     NaN
1161540    18.0
1161541     NaN
Name: Bene_Race_API_Cnt, Length: 1100726, dtype: float64

### Bene_Race_Hspnc_Cnt

In [164]:
# Null Values Present use Imputation
part_b[part_b['Bene_Race_Hspnc_Cnt'].isnull()]["Fraud"].value_counts()

0    830342
1        80
Name: Fraud, dtype: int64

In [165]:
part_b["Bene_Race_Hspnc_Cnt"].value_counts()

0        53175
12        8046
11        7970
13        7868
14        7592
         ...  
1,034        1
951          1
1,118        1
1,221        1
1,080        1
Name: Bene_Race_Hspnc_Cnt, Length: 1099, dtype: int64

In [166]:
part_b["Bene_Race_Hspnc_Cnt"] = part_b["Bene_Race_Hspnc_Cnt"].replace(',', '', regex=True).astype(float)

In [167]:
part_b["Bene_Race_Hspnc_Cnt"]

0           12.0
1           49.0
2            NaN
3            NaN
4            NaN
           ...  
1161536    131.0
1161537     31.0
1161538      NaN
1161540      NaN
1161541      NaN
Name: Bene_Race_Hspnc_Cnt, Length: 1100726, dtype: float64

### Bene_Race_NatInd_Cnt

In [168]:
# Null Values Present use Imputation
part_b[part_b['Bene_Race_NatInd_Cnt'].isnull()]["Fraud"].value_counts()

0    753225
1        80
Name: Fraud, dtype: int64

In [169]:
part_b["Bene_Race_NatInd_Cnt"].value_counts()

0        328214
11         1145
13         1002
12          996
14          895
          ...  
899           1
414           1
798           1
501           1
1,412         1
Name: Bene_Race_NatInd_Cnt, Length: 458, dtype: int64

In [170]:
part_b["Bene_Race_NatInd_Cnt"] = part_b["Bene_Race_NatInd_Cnt"].replace(',', '', regex=True).astype(float)

In [171]:
part_b["Bene_Race_NatInd_Cnt"]

0          NaN
1          NaN
2          0.0
3          NaN
4          NaN
          ... 
1161536    NaN
1161537    NaN
1161538    NaN
1161540    NaN
1161541    0.0
Name: Bene_Race_NatInd_Cnt, Length: 1100726, dtype: float64

### Bene_Race_Othr_Cnt

In [172]:
# Null Values Present use Imputation
part_b[part_b['Bene_Race_Othr_Cnt'].isnull()]["Fraud"].value_counts()

0    917687
1        98
Name: Fraud, dtype: int64

In [173]:
part_b["Bene_Race_Othr_Cnt"].value_counts()

0      63473
11      7424
12      7265
13      6884
14      6370
       ...  
530        1
332        1
317        1
536        1
328        1
Name: Bene_Race_Othr_Cnt, Length: 366, dtype: int64

In [174]:
part_b["Bene_Race_Othr_Cnt"] = part_b["Bene_Race_Othr_Cnt"].replace(',', '', regex=True).astype(float)

In [175]:
part_b["Bene_Race_Othr_Cnt"]

0            NaN
1          128.0
2            NaN
3            NaN
4            NaN
           ...  
1161536      NaN
1161537      NaN
1161538      NaN
1161540     17.0
1161541      NaN
Name: Bene_Race_Othr_Cnt, Length: 1100726, dtype: float64

### Bene_Dual_Cnt

In [176]:
# Null Values Present use Imputation
part_b[part_b['Bene_Dual_Cnt'].isnull()]["Fraud"].value_counts()

0    318169
1        31
Name: Fraud, dtype: int64

In [177]:
part_b["Bene_Dual_Cnt"].value_counts()

0        36092
11       18505
12       17364
13       16877
14       16284
         ...  
2,020        1
1,984        1
2,698        1
1,573        1
1,939        1
Name: Bene_Dual_Cnt, Length: 1971, dtype: int64

In [178]:
part_b["Bene_Dual_Cnt"] = part_b["Bene_Dual_Cnt"].replace(',', '', regex=True).astype(float)

In [179]:
part_b["Bene_Dual_Cnt"]

0           61.0
1          125.0
2           59.0
3           11.0
4           38.0
           ...  
1161536    127.0
1161537     35.0
1161538      NaN
1161540     44.0
1161541     85.0
Name: Bene_Dual_Cnt, Length: 1100726, dtype: float64

### Bene_Ndual_Cnt

In [180]:
# Null Values Present use Imputation
part_b[part_b['Bene_Ndual_Cnt'].isnull()]["Fraud"].value_counts()

0    318169
1        31
Name: Fraud, dtype: int64

In [181]:
part_b["Bene_Ndual_Cnt"].value_counts()

11        5496
12        5166
13        4767
14        4582
16        4446
          ... 
3,846        1
3,673        1
15,668       1
3,120        1
6,423        1
Name: Bene_Ndual_Cnt, Length: 4677, dtype: int64

In [182]:
part_b["Bene_Ndual_Cnt"] = part_b["Bene_Ndual_Cnt"].replace(',', '', regex=True).astype(float)

In [183]:
part_b["Bene_Ndual_Cnt"]

0           230.0
1          2508.0
2           108.0
3            45.0
4            51.0
            ...  
1161536      28.0
1161537     298.0
1161538       NaN
1161540     412.0
1161541     322.0
Name: Bene_Ndual_Cnt, Length: 1100726, dtype: float64

### Bene_CC_AF_Pct

In [186]:
# Null Values Present use Imputation
part_b[part_b['Bene_CC_AF_Pct'].isnull()]["Fraud"].value_counts()

0    378624
1        52
Name: Fraud, dtype: int64

In [187]:
part_b["Bene_CC_AF_Pct"].value_counts()

0%        51829
10%       46896
11.00%    46082
9.00%     42781
12.00%    42015
          ...  
73.00%       54
74.00%       42
72.00%       41
2.00%        33
1.0%          1
Name: Bene_CC_AF_Pct, Length: 76, dtype: int64

In [190]:
part_b["Bene_CC_AF_Pct"] = part_b["Bene_CC_AF_Pct"].replace('%', '', regex=True).astype(float)

In [191]:
part_b["Bene_CC_AF_Pct"]

0          31.0
1          10.0
2           8.0
3           NaN
4           NaN
           ... 
1161536    24.0
1161537    14.0
1161538     NaN
1161540    12.0
1161541    30.0
Name: Bene_CC_AF_Pct, Length: 1100726, dtype: float64

### Bene_CC_Alzhmr_Pct

In [192]:
# Null Values Present use Imputation
part_b[part_b['Bene_CC_Alzhmr_Pct'].isnull()]["Fraud"].value_counts()

0    379013
1        46
Name: Fraud, dtype: int64

In [193]:
part_b["Bene_CC_Alzhmr_Pct"].value_counts()

0%        47371
8.00%     38798
9.00%     38016
7.00%     35497
10%       34999
          ...  
73.00%     1027
72.00%      987
74.00%      929
2.00%       278
1.0%          2
Name: Bene_CC_Alzhmr_Pct, Length: 76, dtype: int64

In [194]:
part_b["Bene_CC_Alzhmr_Pct"] = part_b["Bene_CC_Alzhmr_Pct"].replace('%', '', regex=True).astype(float)

In [195]:
part_b["Bene_CC_Alzhmr_Pct"]

0          43.0
1           6.0
2           NaN
3           NaN
4          13.0
           ... 
1161536    54.0
1161537    12.0
1161538     NaN
1161540    10.0
1161541    43.0
Name: Bene_CC_Alzhmr_Pct, Length: 1100726, dtype: float64

### Bene_CC_Asthma_Pct

In [196]:
# Null Values Present use Imputation
part_b[part_b['Bene_CC_Asthma_Pct'].isnull()]["Fraud"].value_counts()

0    473689
1        59
Name: Fraud, dtype: int64

In [197]:
part_b["Bene_CC_Asthma_Pct"].value_counts()

8.00%     67589
9.00%     66940
10%       61426
7.00%     61358
11.00%    52285
          ...  
63.00%        6
72.00%        6
70.0%         5
74.00%        4
1.0%          3
Name: Bene_CC_Asthma_Pct, Length: 76, dtype: int64

In [198]:
part_b["Bene_CC_Asthma_Pct"] = part_b["Bene_CC_Asthma_Pct"].replace('%', '', regex=True).astype(float)

In [199]:
part_b["Bene_CC_Asthma_Pct"]

0          15.0
1           4.0
2          13.0
3           NaN
4          20.0
           ... 
1161536     8.0
1161537     5.0
1161538     NaN
1161540     9.0
1161541    13.0
Name: Bene_CC_Asthma_Pct, Length: 1100726, dtype: float64

### Bene_CC_Cncr_Pct

In [200]:
# Null Values Present use Imputation
part_b[part_b['Bene_CC_Cncr_Pct'].isnull()]["Fraud"].value_counts()

0    387680
1        59
Name: Fraud, dtype: int64

In [201]:
part_b["Bene_CC_Cncr_Pct"].value_counts()

11.00%    62465
12.00%    59683
10%       59072
13.00%    57537
9.00%     50585
          ...  
68.00%      352
64.00%      348
72.00%      347
3.00%       174
2.00%        16
Name: Bene_CC_Cncr_Pct, Length: 75, dtype: int64

In [202]:
part_b["Bene_CC_Cncr_Pct"] = part_b["Bene_CC_Cncr_Pct"].replace('%', '', regex=True).astype(float)

In [203]:
part_b["Bene_CC_Cncr_Pct"]

0          18.0
1          14.0
2          10.0
3           NaN
4           NaN
           ... 
1161536    12.0
1161537    10.0
1161538     NaN
1161540     9.0
1161541    22.0
Name: Bene_CC_Cncr_Pct, Length: 1100726, dtype: float64

### Bene_CC_CHF_Pct

In [204]:
# Null Values Present use Imputation
part_b[part_b['Bene_CC_CHF_Pct'].isnull()]["Fraud"].value_counts()

0    290372
1        37
Name: Fraud, dtype: int64

In [205]:
part_b["Bene_CC_CHF_Pct"].value_counts()

13.00%    30195
14.00%    29597
15.00%    29371
16.00%    28084
12.00%    26404
          ...  
72.00%      893
74.00%      753
4.00%       465
3.00%        97
2.00%         4
Name: Bene_CC_CHF_Pct, Length: 75, dtype: int64

In [206]:
part_b["Bene_CC_CHF_Pct"] = part_b["Bene_CC_CHF_Pct"].replace('%', '', regex=True).astype(float)

In [207]:
part_b["Bene_CC_CHF_Pct"]

0          48.0
1          12.0
2          21.0
3           NaN
4          20.0
           ... 
1161536    55.0
1161537    15.0
1161538     NaN
1161540    15.0
1161541    55.0
Name: Bene_CC_CHF_Pct, Length: 1100726, dtype: float64

### Bene_CC_CKD_Pct

In [208]:
# Null Values Present use Imputation
part_b[part_b['Bene_CC_CKD_Pct'].isnull()]["Fraud"].value_counts()

0    176881
1        23
Name: Fraud, dtype: int64

In [209]:
part_b["Bene_CC_CKD_Pct"].value_counts()

75.00%    53954
28.00%    23472
27.00%    23019
29.00%    22970
30.0%     22711
          ...  
7.00%        73
6.00%        21
5.00%         4
4.00%         1
3.00%         1
Name: Bene_CC_CKD_Pct, Length: 74, dtype: int64

In [210]:
part_b["Bene_CC_CKD_Pct"] = part_b["Bene_CC_CKD_Pct"].replace('%', '', regex=True).astype(float)

In [211]:
part_b["Bene_CC_CKD_Pct"]

0          65.0
1          22.0
2          34.0
3           NaN
4          31.0
           ... 
1161536    66.0
1161537    34.0
1161538    17.0
1161540    23.0
1161541    75.0
Name: Bene_CC_CKD_Pct, Length: 1100726, dtype: float64

### Bene_CC_COPD_Pct

In [212]:
# Null Values Present use Imputation
part_b[part_b['Bene_CC_COPD_Pct'].isnull()]["Fraud"].value_counts()

0    338761
1        41
Name: Fraud, dtype: int64

In [213]:
part_b["Bene_CC_COPD_Pct"].value_counts()

13.00%    32199
11.00%    31267
12.00%    30586
0%        30447
14.00%    30073
          ...  
72.00%       87
73.00%       82
74.00%       54
2.00%        41
1.0%          2
Name: Bene_CC_COPD_Pct, Length: 76, dtype: int64

In [214]:
part_b["Bene_CC_COPD_Pct"] = part_b["Bene_CC_COPD_Pct"].replace('%', '', regex=True).astype(float)

In [215]:
part_b["Bene_CC_COPD_Pct"]

0          29.0
1           6.0
2          26.0
3           NaN
4          12.0
           ... 
1161536    40.0
1161537     9.0
1161538     NaN
1161540     9.0
1161541    38.0
Name: Bene_CC_COPD_Pct, Length: 1100726, dtype: float64

### Bene_CC_Dprssn_Pct

In [216]:
# Null Values Present use Imputation
part_b[part_b['Bene_CC_Dprssn_Pct'].isnull()]["Fraud"].value_counts()

0    178866
1        30
Name: Fraud, dtype: int64

In [217]:
part_b["Bene_CC_Dprssn_Pct"].value_counts()

75.00%    41380
25.00%    30386
26.00%    30307
24.00%    30178
27.00%    29961
          ...  
6.00%       260
5.00%       131
4.00%        52
3.00%        27
2.00%         5
Name: Bene_CC_Dprssn_Pct, Length: 75, dtype: int64

In [218]:
part_b["Bene_CC_Dprssn_Pct"] = part_b["Bene_CC_Dprssn_Pct"].replace('%', '', regex=True).astype(float)

In [219]:
part_b["Bene_CC_Dprssn_Pct"]

0          35.0
1          15.0
2          37.0
3          27.0
4          43.0
           ... 
1161536    52.0
1161537    28.0
1161538    24.0
1161540    19.0
1161541    41.0
Name: Bene_CC_Dprssn_Pct, Length: 1100726, dtype: float64

### Bene_CC_Dbts_Pct

In [220]:
# Null Values Present use Imputation
part_b[part_b['Bene_CC_Dbts_Pct'].isnull()]["Fraud"].value_counts()

0    180122
1        24
Name: Fraud, dtype: int64

In [221]:
part_b["Bene_CC_Dbts_Pct"].value_counts()

33.00%    31577
31.00%    31331
32.00%    30895
30.0%     30741
29.00%    30206
          ...  
8.00%       187
7.00%        81
6.00%        46
5.00%        18
4.00%         3
Name: Bene_CC_Dbts_Pct, Length: 73, dtype: int64

In [222]:
part_b["Bene_CC_Dbts_Pct"] = part_b["Bene_CC_Dbts_Pct"].replace('%', '', regex=True).astype(float)

In [223]:
part_b["Bene_CC_Dbts_Pct"]

0          47.0
1          20.0
2          35.0
3           NaN
4          39.0
           ... 
1161536    58.0
1161537    26.0
1161538    26.0
1161540    21.0
1161541    48.0
Name: Bene_CC_Dbts_Pct, Length: 1100726, dtype: float64

### Bene_CC_Hyplpdma_Pct

In [224]:
# Null Values Present use Imputation
part_b[part_b['Bene_CC_Hyplpdma_Pct'].isnull()]["Fraud"].value_counts()

0    90438
1       16
Name: Fraud, dtype: int64

In [225]:
part_b["Bene_CC_Hyplpdma_Pct"].value_counts()

75.00%    156698
63.00%     32806
65.00%     32083
67.00%     31829
68.00%     31418
           ...  
14.00%        43
13.00%        13
11.00%         7
12.00%         6
10%            3
Name: Bene_CC_Hyplpdma_Pct, Length: 67, dtype: int64

In [226]:
part_b["Bene_CC_Hyplpdma_Pct"] = part_b["Bene_CC_Hyplpdma_Pct"].replace('%', '', regex=True).astype(float)

In [227]:
part_b["Bene_CC_Hyplpdma_Pct"]

0          73.0
1          48.0
2          52.0
3          52.0
4          43.0
           ... 
1161536    71.0
1161537    49.0
1161538    61.0
1161540    44.0
1161541    75.0
Name: Bene_CC_Hyplpdma_Pct, Length: 1100726, dtype: float64

### Bene_CC_Hyprtnsn_Pct

In [228]:
# Null Values Present use Imputation
part_b[part_b['Bene_CC_Hyprtnsn_Pct'].isnull()]["Fraud"].value_counts()

0    68403
1       12
Name: Fraud, dtype: int64

In [229]:
part_b["Bene_CC_Hyprtnsn_Pct"].value_counts()

75.00%    463852
71.00%     26722
73.00%     26458
74.00%     25983
68.00%     25549
           ...  
17.00%         6
15.00%         5
14.00%         4
13.00%         4
9.00%          2
Name: Bene_CC_Hyprtnsn_Pct, Length: 65, dtype: int64

In [230]:
part_b["Bene_CC_Hyprtnsn_Pct"] = part_b["Bene_CC_Hyprtnsn_Pct"].replace('%', '', regex=True).astype(float)

In [231]:
part_b["Bene_CC_Hyprtnsn_Pct"]

0          75.0
1          48.0
2          68.0
3          50.0
4          60.0
           ... 
1161536    75.0
1161537    66.0
1161538    53.0
1161540    53.0
1161541    75.0
Name: Bene_CC_Hyprtnsn_Pct, Length: 1100726, dtype: float64

### Bene_CC_IHD_Pct

In [232]:
# Null Values Present use Imputation
part_b[part_b['Bene_CC_IHD_Pct'].isnull()]["Fraud"].value_counts()

0    181276
1        26
Name: Fraud, dtype: int64

In [233]:
part_b["Bene_CC_IHD_Pct"].value_counts()

75.00%    27175
33.00%    26541
31.00%    26121
30.0%     25759
32.00%    25534
          ...  
7.00%        53
6.00%        31
5.00%         9
4.00%         3
3.00%         1
Name: Bene_CC_IHD_Pct, Length: 74, dtype: int64

In [234]:
part_b["Bene_CC_IHD_Pct"] = part_b["Bene_CC_IHD_Pct"].replace('%', '', regex=True).astype(float)

In [235]:
part_b["Bene_CC_IHD_Pct"]

0          66.0
1          24.0
2          31.0
3          23.0
4          33.0
           ... 
1161536    70.0
1161537    25.0
1161538    26.0
1161540    26.0
1161541    61.0
Name: Bene_CC_IHD_Pct, Length: 1100726, dtype: float64

### Bene_CC_Opo_Pct

In [236]:
# Null Values Present use Imputation
part_b[part_b['Bene_CC_Opo_Pct'].isnull()]["Fraud"].value_counts()

0    439811
1        57
Name: Fraud, dtype: int64

In [237]:
part_b["Bene_CC_Opo_Pct"].value_counts()

10%       68599
9.00%     68255
11.00%    62782
8.00%     60822
12.00%    53053
          ...  
68.00%       14
73.00%        8
72.00%        7
74.00%        5
1.0%          4
Name: Bene_CC_Opo_Pct, Length: 76, dtype: int64

In [238]:
part_b["Bene_CC_Opo_Pct"] = part_b["Bene_CC_Opo_Pct"].replace('%', '', regex=True).astype(float)

In [239]:
part_b["Bene_CC_Opo_Pct"]

0          11.0
1          10.0
2           NaN
3          20.0
4          13.0
           ... 
1161536     9.0
1161537     9.0
1161538     NaN
1161540     9.0
1161541    11.0
Name: Bene_CC_Opo_Pct, Length: 1100726, dtype: float64

### Bene_CC_RAOA_Pct

In [240]:
# Null Values Present use Imputation
part_b[part_b['Bene_CC_RAOA_Pct'].isnull()]["Fraud"].value_counts()

0    115750
1        19
Name: Fraud, dtype: int64

In [241]:
part_b["Bene_CC_RAOA_Pct"].value_counts()

75.00%    67447
44.00%    36138
46.00%    35461
48.00%    35345
45.00%    35177
          ...  
9.00%        23
8.00%        10
7.00%         4
6.00%         1
4.00%         1
Name: Bene_CC_RAOA_Pct, Length: 72, dtype: int64

In [242]:
part_b["Bene_CC_RAOA_Pct"] = part_b["Bene_CC_RAOA_Pct"].replace('%', '', regex=True).astype(float)

In [243]:
part_b["Bene_CC_RAOA_Pct"]

0          51.0
1          38.0
2          75.0
3          32.0
4          57.0
           ... 
1161536    48.0
1161537    41.0
1161538    56.0
1161540    37.0
1161541    48.0
Name: Bene_CC_RAOA_Pct, Length: 1100726, dtype: float64

### Bene_CC_Sz_Pct

In [244]:
# Null Values Present use Imputation
part_b[part_b['Bene_CC_Sz_Pct'].isnull()]["Fraud"].value_counts()

0    625252
1        61
Name: Fraud, dtype: int64

In [245]:
part_b["Bene_CC_Sz_Pct"].value_counts()

0%        182760
3.00%      28362
4.00%      28035
5.00%      25656
6.00%      23623
           ...  
73.00%       339
74.00%       318
70.0%        314
72.00%       297
66.00%       277
Name: Bene_CC_Sz_Pct, Length: 76, dtype: int64

In [246]:
part_b["Bene_CC_Sz_Pct"] = part_b["Bene_CC_Sz_Pct"].replace('%', '', regex=True).astype(float)

In [247]:
part_b["Bene_CC_Sz_Pct"]

0           9.0
1           1.0
2           NaN
3           0.0
4           NaN
           ... 
1161536    27.0
1161537     NaN
1161538     NaN
1161540     NaN
1161541     7.0
Name: Bene_CC_Sz_Pct, Length: 1100726, dtype: float64

### Bene_CC_Strok_Pct

In [248]:
# Null Values Present use Imputation
part_b[part_b['Bene_CC_Strok_Pct'].isnull()]["Fraud"].value_counts()

0    532691
1        60
Name: Fraud, dtype: int64

In [249]:
part_b["Bene_CC_Strok_Pct"].value_counts()

0%        107214
4.00%      43820
5.00%      42907
6.00%      36678
7.00%      31577
           ...  
71.00%       117
73.00%       113
70.0%        108
74.00%        98
72.00%        96
Name: Bene_CC_Strok_Pct, Length: 76, dtype: int64

In [250]:
part_b["Bene_CC_Strok_Pct"] = part_b["Bene_CC_Strok_Pct"].replace('%', '', regex=True).astype(float)

In [251]:
part_b["Bene_CC_Strok_Pct"]

0          19.0
1           3.0
2           NaN
3           NaN
4           NaN
           ... 
1161536    15.0
1161537     4.0
1161538     NaN
1161540     2.0
1161541    23.0
Name: Bene_CC_Strok_Pct, Length: 1100726, dtype: float64

### Bene_Avg_Risk_Scre

In [252]:
# Null Values Present use Imputation
part_b[part_b['Bene_Avg_Risk_Scre'].isnull()]["Fraud"].value_counts()

Series([], Name: Fraud, dtype: int64)

In [253]:
part_b["Bene_Avg_Risk_Scre"].value_counts()

1.1567    125
1.1256    124
1.0264    124
1.0714    123
1.0378    122
         ... 
4.1688      1
8.8134      1
4.1579      1
6.4915      1
4.3047      1
Name: Bene_Avg_Risk_Scre, Length: 49042, dtype: int64

In [254]:
part_b["Bene_Avg_Risk_Scre"] = part_b["Bene_Avg_Risk_Scre"].astype(float)

In [255]:
part_b["Bene_Avg_Risk_Scre"]

0          2.5028
1          1.1124
2          1.6935
3          0.7089
4          2.1137
            ...  
1161536    3.4526
1161537    1.2418
1161538    0.9525
1161540    1.1737
1161541    2.7672
Name: Bene_Avg_Risk_Scre, Length: 1100726, dtype: float64

### Rndrng_Prvdr_Crdntls

In [261]:
# Null Values Present use Imputation
part_b[part_b['Rndrng_Prvdr_Crdntls'].isnull()]["Fraud"].value_counts()

0    71535
1        5
Name: Fraud, dtype: int64

In [262]:
part_b["Rndrng_Prvdr_Crdntls"].value_counts()

M.D.                    251923
MD                      251357
PA-C                     44840
CRNA                     37011
D.O.                     35005
                         ...  
PT, MSPT,  ATC, CSCS         1
DC, DACRB, CSCS              1
RN MS PMHNP                  1
PA AA                        1
DMD, MD, MHS                 1
Name: Rndrng_Prvdr_Crdntls, Length: 17582, dtype: int64

### Rndrng_Prvdr_Gndr

In [265]:
# Null Values Present use Imputation
part_b[part_b['Rndrng_Prvdr_Gndr'].isnull()]["Fraud"].value_counts()

Series([], Name: Fraud, dtype: int64)

In [266]:
part_b["Rndrng_Prvdr_Gndr"].value_counts()

M    582943
F    517783
Name: Rndrng_Prvdr_Gndr, dtype: int64

In [268]:
part_b["Rndrng_Prvdr_Gndr"] = part_b["Rndrng_Prvdr_Gndr"].apply(lambda row : 1 if row == 'F' else 0)

In [269]:
part_b["Rndrng_Prvdr_Gndr"] = part_b["Rndrng_Prvdr_Gndr"].astype(float)

In [270]:
part_b["Rndrng_Prvdr_Gndr"]

0          0.0
1          0.0
2          0.0
3          1.0
4          0.0
          ... 
1161536    0.0
1161537    1.0
1161538    0.0
1161540    0.0
1161541    1.0
Name: Rndrng_Prvdr_Gndr, Length: 1100726, dtype: float64

### Rndrng_Prvdr_Ent_Cd

In [272]:
# Null Values Present use Imputation
part_b[part_b['Rndrng_Prvdr_Ent_Cd'].isnull()]["Fraud"].value_counts()

Series([], Name: Fraud, dtype: int64)

In [273]:
part_b["Rndrng_Prvdr_Ent_Cd"].value_counts() # drop it

I    1100726
Name: Rndrng_Prvdr_Ent_Cd, dtype: int64

In [275]:
part_b = part_b.drop("Rndrng_Prvdr_Ent_Cd",axis=1)

In [257]:
part_b.columns

Index(['Rndrng_Prvdr_Last_Org_Name', 'Rndrng_Prvdr_First_Name',
       'Rndrng_Prvdr_MI', 'Rndrng_Prvdr_Crdntls', 'Rndrng_Prvdr_Gndr',
       'Rndrng_Prvdr_Ent_Cd', 'Rndrng_Prvdr_St1', 'Rndrng_Prvdr_St2',
       'Rndrng_Prvdr_City', 'Rndrng_Prvdr_State_Abrvtn',
       'Rndrng_Prvdr_State_FIPS', 'Rndrng_Prvdr_Zip5', 'Rndrng_Prvdr_RUCA',
       'Rndrng_Prvdr_RUCA_Desc', 'Rndrng_Prvdr_Cntry', 'Rndrng_Prvdr_Type',
       'Rndrng_Prvdr_Mdcr_Prtcptg_Ind', 'Tot_HCPCS_Cds', 'Tot_Benes',
       'Tot_Srvcs', 'Tot_Sbmtd_Chrg', 'Tot_Mdcr_Alowd_Amt',
       'Tot_Mdcr_Pymt_Amt', 'Tot_Mdcr_Stdzd_Amt', 'Drug_Sprsn_Ind',
       'Drug_Tot_HCPCS_Cds', 'Drug_Tot_Benes', 'Drug_Tot_Srvcs',
       'Drug_Sbmtd_Chrg', 'Drug_Mdcr_Alowd_Amt', 'Drug_Mdcr_Pymt_Amt',
       'Drug_Mdcr_Stdzd_Amt', 'Med_Sprsn_Ind', 'Med_Tot_HCPCS_Cds',
       'Med_Tot_Benes', 'Med_Tot_Srvcs', 'Med_Sbmtd_Chrg',
       'Med_Mdcr_Alowd_Amt', 'Med_Mdcr_Pymt_Amt', 'Med_Mdcr_Stdzd_Amt',
       'Bene_Avg_Age', 'Bene_Age_LT_65_Cnt', 'Bene_Ag

In [259]:
drop_cols = ['Rndrng_Prvdr_Last_Org_Name', 'Rndrng_Prvdr_First_Name',
       'Rndrng_Prvdr_MI','Rndrng_Prvdr_St1', 'Rndrng_Prvdr_St2',
       'Rndrng_Prvdr_City', 'Rndrng_Prvdr_State_Abrvtn',
       'Rndrng_Prvdr_State_FIPS', 'Rndrng_Prvdr_Zip5', 'Rndrng_Prvdr_RUCA',
       'Rndrng_Prvdr_RUCA_Desc', 'Rndrng_Prvdr_Cntry']
part_b = part_b.drop(drop_cols,axis=1)

In [276]:
part_b.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1100726 entries, 0 to 1161541
Data columns (total 62 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   Rndrng_Prvdr_Crdntls           1029186 non-null  object 
 1   Rndrng_Prvdr_Gndr              1100726 non-null  float64
 2   Rndrng_Prvdr_Type              1100726 non-null  object 
 3   Rndrng_Prvdr_Mdcr_Prtcptg_Ind  1100726 non-null  int64  
 4   Tot_HCPCS_Cds                  1100726 non-null  float64
 5   Tot_Benes                      1100726 non-null  float64
 6   Tot_Srvcs                      1100726 non-null  float64
 7   Tot_Sbmtd_Chrg                 1100726 non-null  float64
 8   Tot_Mdcr_Alowd_Amt             1100726 non-null  float64
 9   Tot_Mdcr_Pymt_Amt              1100726 non-null  float64
 10  Tot_Mdcr_Stdzd_Amt             1100726 non-null  float64
 11  Drug_Sprsn_Ind                 130700 non-null   float64
 12  Drug_Tot_HCPCS

In [278]:
part_b

,Rndrng_Prvdr_Crdntls,Rndrng_Prvdr_Gndr,Rndrng_Prvdr_Type,Rndrng_Prvdr_Mdcr_Prtcptg_Ind,Tot_HCPCS_Cds,Tot_Benes,Tot_Srvcs,Tot_Sbmtd_Chrg,Tot_Mdcr_Alowd_Amt,Tot_Mdcr_Pymt_Amt,...,Bene_CC_Hyprtnsn_Pct,Bene_CC_IHD_Pct,Bene_CC_Opo_Pct,Bene_CC_RAOA_Pct,Bene_CC_Sz_Pct,Bene_CC_Strok_Pct,Bene_Avg_Risk_Scre,Rndrng_NPI,Fraud,FraudType
0,M.D.,0.0,Internal Medicine,1,16.0,291.0,764.0,402812.00,85319.63,69175.78,...,75.0,66.0,11.0,51.0,9.0,19.0,2.5028,1003000126,0,0
1,M.D.,0.0,Pathology,1,18.0,2633.0,5930.0,915291.00,227372.53,176497.74,...,48.0,24.0,10.0,38.0,1.0,3.0,1.1124,1003000134,0,0
2,M.D.,0.0,Anesthesiology,1,54.0,167.0,2003.0,182532.48,101757.15,76938.82,...,68.0,31.0,NaN,75.0,NaN,NaN,1.6935,1003000142,0,0
3,M.D.,1.0,Obstetrics & Gynecology,1,21.0,56.0,571.0,23680.00,9011.99,7224.35,...,50.0,23.0,20.0,32.0,0.0,NaN,0.7089,1003000423,0,0
4,MD,0.0,General Surgery,1,30.0,89.0,125.0,152154.00,30631.10,23962.85,...,60.0,33.0,13.0,57.0,NaN,NaN,2.1137,1003000480,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1161536,M.D.,0.0,Emergency Medicine,1,9.0,155.0,171.0,329504.00,32266.94,25803.90,...,75.0,70.0,9.0,48.0,27.0,15.0,3.4526,1992999437,0,0
1161537,M.D.,1.0,Internal Medicine,1,40.0,333.0,1093.0,182515.00,93505.14,68252.03,...,66.0,25.0,9.0,41.0,NaN,4.0,1.2418,1992999551,0,0
1161538,D.C.,0.0,Chiropractic,1,2.0,66.0,415.0,20985.00,16171.02,11759.42,...,53.0,26.0,NaN,56.0,NaN,NaN,0.9525,1992999585,0,0
1161540,M.D.,0.0,Otolaryngology,1,46.0,456.0,851.0,304460.56,113283.09,85463.83,...,53.0,26.0,9.0,37.0,NaN,2.0,1.1737,1992999825,0,0


In [280]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
part_b['Rndrng_Prvdr_Type'] = label_encoder.fit_transform(part_b['Rndrng_Prvdr_Type'])

In [281]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
part_b['Rndrng_Prvdr_Crdntls'] = label_encoder.fit_transform(part_b['Rndrng_Prvdr_Crdntls'])

In [295]:
part_b['FraudType'] = part_b['FraudType'].astype(str)

In [296]:
from sklearn.preprocessing import LabelEncoder
label_encoder1 = LabelEncoder()
part_b['FraudType'] = label_encoder1.fit_transform(part_b['FraudType'])

In [298]:
part_b['FraudType'].value_counts()

0     1100610
10         35
1          30
7          14
2          13
5           8
11          8
8           2
4           2
12          1
3           1
9           1
6           1
Name: FraudType, dtype: int64

In [299]:
part_b.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1100726 entries, 0 to 1161541
Data columns (total 62 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   Rndrng_Prvdr_Crdntls           1100726 non-null  int64  
 1   Rndrng_Prvdr_Gndr              1100726 non-null  float64
 2   Rndrng_Prvdr_Type              1100726 non-null  int64  
 3   Rndrng_Prvdr_Mdcr_Prtcptg_Ind  1100726 non-null  int64  
 4   Tot_HCPCS_Cds                  1100726 non-null  float64
 5   Tot_Benes                      1100726 non-null  float64
 6   Tot_Srvcs                      1100726 non-null  float64
 7   Tot_Sbmtd_Chrg                 1100726 non-null  float64
 8   Tot_Mdcr_Alowd_Amt             1100726 non-null  float64
 9   Tot_Mdcr_Pymt_Amt              1100726 non-null  float64
 10  Tot_Mdcr_Stdzd_Amt             1100726 non-null  float64
 11  Drug_Sprsn_Ind                 130700 non-null   float64
 12  Drug_Tot_HCPCS

In [305]:
part_b.columns

Index(['Rndrng_Prvdr_Crdntls', 'Rndrng_Prvdr_Gndr', 'Rndrng_Prvdr_Type',
       'Rndrng_Prvdr_Mdcr_Prtcptg_Ind', 'Tot_HCPCS_Cds', 'Tot_Benes',
       'Tot_Srvcs', 'Tot_Sbmtd_Chrg', 'Tot_Mdcr_Alowd_Amt',
       'Tot_Mdcr_Pymt_Amt', 'Tot_Mdcr_Stdzd_Amt', 'Drug_Sprsn_Ind',
       'Drug_Tot_HCPCS_Cds', 'Drug_Tot_Benes', 'Drug_Tot_Srvcs',
       'Drug_Sbmtd_Chrg', 'Drug_Mdcr_Alowd_Amt', 'Drug_Mdcr_Pymt_Amt',
       'Drug_Mdcr_Stdzd_Amt', 'Med_Sprsn_Ind', 'Med_Tot_HCPCS_Cds',
       'Med_Tot_Benes', 'Med_Tot_Srvcs', 'Med_Sbmtd_Chrg',
       'Med_Mdcr_Alowd_Amt', 'Med_Mdcr_Pymt_Amt', 'Med_Mdcr_Stdzd_Amt',
       'Bene_Avg_Age', 'Bene_Age_LT_65_Cnt', 'Bene_Age_65_74_Cnt',
       'Bene_Age_75_84_Cnt', 'Bene_Age_GT_84_Cnt', 'Bene_Feml_Cnt',
       'Bene_Male_Cnt', 'Bene_Race_Wht_Cnt', 'Bene_Race_Black_Cnt',
       'Bene_Race_API_Cnt', 'Bene_Race_Hspnc_Cnt', 'Bene_Race_NatInd_Cnt',
       'Bene_Race_Othr_Cnt', 'Bene_Dual_Cnt', 'Bene_Ndual_Cnt',
       'Bene_CC_AF_Pct', 'Bene_CC_Alzhmr_Pct', 'Bene

In [328]:
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler
import statsmodels.imputation.mice as mice
from sklearn.impute import KNNImputer

feature_cols = ['Rndrng_Prvdr_Crdntls', 'Rndrng_Prvdr_Gndr', 'Rndrng_Prvdr_Type',
       'Rndrng_Prvdr_Mdcr_Prtcptg_Ind', 'Tot_HCPCS_Cds', 'Tot_Benes',
       'Tot_Srvcs', 'Tot_Sbmtd_Chrg', 'Tot_Mdcr_Alowd_Amt',
       'Tot_Mdcr_Pymt_Amt','Tot_Mdcr_Stdzd_Amt', 'Drug_Sprsn_Ind',
       'Drug_Tot_HCPCS_Cds', 'Drug_Tot_Benes', 'Drug_Tot_Srvcs',
       'Drug_Sbmtd_Chrg', 'Drug_Mdcr_Alowd_Amt', 'Drug_Mdcr_Pymt_Amt',
       'Drug_Mdcr_Stdzd_Amt', 'Med_Sprsn_Ind','Med_Tot_HCPCS_Cds',
       'Med_Tot_Benes', 'Med_Tot_Srvcs']

part_b_imputed = part_b.copy()

# Create an imputer
imputer = IterativeImputer(max_iter=10, random_state=42)

# Perform imputation
imputed_values = imputer.fit_transform(part_b_imputed[feature_cols])

# Convert the imputed values back to a DataFrame
print(imputed_values.shape)

(1100726, 23)


In [311]:
# Assign the imputed values back to the DataFrame
part_b_imputed[feature_cols] = imputed_values
print(part_b_imputed[feature_cols].isnull().sum())
part_b_imputed[feature_cols]

Rndrng_Prvdr_Crdntls             0
Rndrng_Prvdr_Gndr                0
Rndrng_Prvdr_Type                0
Rndrng_Prvdr_Mdcr_Prtcptg_Ind    0
Tot_HCPCS_Cds                    0
Tot_Benes                        0
Tot_Srvcs                        0
Tot_Sbmtd_Chrg                   0
Tot_Mdcr_Alowd_Amt               0
Tot_Mdcr_Pymt_Amt                0
dtype: int64


,Rndrng_Prvdr_Crdntls,Rndrng_Prvdr_Gndr,Rndrng_Prvdr_Type,Rndrng_Prvdr_Mdcr_Prtcptg_Ind,Tot_HCPCS_Cds,Tot_Benes,Tot_Srvcs,Tot_Sbmtd_Chrg,Tot_Mdcr_Alowd_Amt,Tot_Mdcr_Pymt_Amt
0,7310.0,0.0,39.0,1.0,16.0,291.0,764.0,402812.00,85319.63,69175.78
1,7310.0,0.0,65.0,1.0,18.0,2633.0,5930.0,915291.00,227372.53,176497.74
2,7310.0,0.0,5.0,1.0,54.0,167.0,2003.0,182532.48,101757.15,76938.82
3,7310.0,1.0,56.0,1.0,21.0,56.0,571.0,23680.00,9011.99,7224.35
4,9043.0,0.0,27.0,1.0,30.0,89.0,125.0,152154.00,30631.10,23962.85
...,...,...,...,...,...,...,...,...,...,...
1161536,7310.0,0.0,22.0,1.0,9.0,155.0,171.0,329504.00,32266.94,25803.90
1161537,7310.0,1.0,39.0,1.0,40.0,333.0,1093.0,182515.00,93505.14,68252.03
1161538,3073.0,0.0,13.0,1.0,2.0,66.0,415.0,20985.00,16171.02,11759.42
1161540,7310.0,0.0,63.0,1.0,46.0,456.0,851.0,304460.56,113283.09,85463.83


In [312]:
print(part_b[feature_cols].isnull().sum())
part_b[feature_cols]

Rndrng_Prvdr_Crdntls             0
Rndrng_Prvdr_Gndr                0
Rndrng_Prvdr_Type                0
Rndrng_Prvdr_Mdcr_Prtcptg_Ind    0
Tot_HCPCS_Cds                    0
Tot_Benes                        0
Tot_Srvcs                        0
Tot_Sbmtd_Chrg                   0
Tot_Mdcr_Alowd_Amt               0
Tot_Mdcr_Pymt_Amt                0
dtype: int64


,Rndrng_Prvdr_Crdntls,Rndrng_Prvdr_Gndr,Rndrng_Prvdr_Type,Rndrng_Prvdr_Mdcr_Prtcptg_Ind,Tot_HCPCS_Cds,Tot_Benes,Tot_Srvcs,Tot_Sbmtd_Chrg,Tot_Mdcr_Alowd_Amt,Tot_Mdcr_Pymt_Amt
0,7310,0.0,39,1,16.0,291.0,764.0,402812.00,85319.63,69175.78
1,7310,0.0,65,1,18.0,2633.0,5930.0,915291.00,227372.53,176497.74
2,7310,0.0,5,1,54.0,167.0,2003.0,182532.48,101757.15,76938.82
3,7310,1.0,56,1,21.0,56.0,571.0,23680.00,9011.99,7224.35
4,9043,0.0,27,1,30.0,89.0,125.0,152154.00,30631.10,23962.85
...,...,...,...,...,...,...,...,...,...,...
1161536,7310,0.0,22,1,9.0,155.0,171.0,329504.00,32266.94,25803.90
1161537,7310,1.0,39,1,40.0,333.0,1093.0,182515.00,93505.14,68252.03
1161538,3073,0.0,13,1,2.0,66.0,415.0,20985.00,16171.02,11759.42
1161540,7310,0.0,63,1,46.0,456.0,851.0,304460.56,113283.09,85463.83


In [300]:
part_b.to_csv("processed_partb.csv")

In [ ]:
part_b_imputed

In [ ]:
part_b_imputed.to_csv("imputedKNN5.csv")

In [ ]:
part_b_imputed[part_b_imputed['Bene_CC_CHF_Pct'].isnull()]["Fraud"].value_counts()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# List of features with missing values
features_with_missing = part_b.columns

# Plotting
data = []
for feature in features_with_missing:
    data.append(part_b[part_b[feature].isnull()]["Fraud"].value_counts())

In [ ]:
# Create a new DataFrame
df = pd.DataFrame()

# Extract feature names, "Fraud" counts, and "Non-Fraud" counts
feature_names = [s.name for s in data]
fraud_counts = [s.iloc[1] if len(s) == 2 else 0 for s in data]
non_fraud_counts = [s.iloc[0] if len(s) == 2 else s.iloc[0] if len(s) == 1 else 0 for s in data]

# Add columns to the new DataFrame
df['Feature'] = feature_names
df['Fraud'] = fraud_counts
df['Non-Fraud'] = non_fraud_counts

In [ ]:
df['Feature'] = range(1, len(part_b.columns) + 1)
df

In [ ]:
# Create two subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8), sharex=True)

# Plot for Fraud counts
ax1.bar(df['Feature'], df['Fraud'], color='black', label='Fraud')
ax1.set_ylabel('Fraud Count')
ax1.set_title('Fraud Counts for Each Feature')

# Plot for Non-Fraud counts
ax2.bar(df['Feature'], df['Non-Fraud'], color='gray', label='Non-Fraud')
ax2.set_xlabel('Part B data features (columns)')
ax2.set_ylabel('Non-Fraud Count')
ax2.set_title('Non-Fraud Counts for Each Feature')

# Adjust layout to prevent clipping of ylabel
plt.tight_layout()

plt.show()

In [ ]:
part_b.dropna(axis=0)

### SAVE ALL THESE AS .CSV FOR PROCESSING FURTHER

In [ ]:
part_b.columns

In [ ]:
part_b_features = part_b[["Rndrng_NPI","Rndrng_Prvdr_Type","Rndrng_Prvdr_Gndr","Tot_Srvcs","Tot_Benes",
        "Tot_Sbmtd_Chrg","Tot_Mdcr_Alowd_Amt","Tot_Mdcr_Pymt_Amt","Tot_Mdcr_Stdzd_Amt","Fraud","FraudType"]].dropna(axis=0)

In [ ]:
part_b_features.info()

In [ ]:
part_b_features.to_csv('labelled_data/part_B/partb_features_data.csv')

In [ ]:
part_b_features["Fraud"].value_counts()